In [3]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import keras.backend as K

# ## Get the dataset and prepare it for analysis and model

# #### Set the index to date

df = pd.read_csv('USD_INR.csv')
df['Date'] = pd.to_datetime(df["Date"])
df_idx = df.set_index(["Date"], drop=True)
# print(df_idx.head(5))


# #### Flip the dataframe

df_idx = df_idx.sort_index(axis=1, ascending=True)
df_idx = df_idx.iloc[::-1]


# #### Plot the data

data = df_idx[['Price']]
data.plot(y='Price')

diff = data.index.values[-1] - data.index.values[0]
days = diff.astype('timedelta64[D]')
days = days / np.timedelta64(1, 'D')
years = int(days/365)
print("Total data: %d years"%years)
print("80 percent data = 1980 to %d"%(1980 + int(0.8*years)))


# #### Create training and testing data

split_date = pd.Timestamp('01-01-2009')

train = data.loc[:split_date]
test = data.loc[split_date:]

# ax = train.plot(figsize=(10,12))
# test.plot(ax=ax)
# plt.legend(['train', 'test'])
# plt.show()


# #### Normalize the dataset

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
train_sc = sc.fit_transform(train)
test_sc = sc.transform(test)


train_sc_df = pd.DataFrame(train_sc, columns=['Y'], index=train.index)
test_sc_df = pd.DataFrame(test_sc, columns=['Y'], index=test.index)

for s in range(1,2):
    train_sc_df['X_{}'.format(s)] = train_sc_df['Y'].shift(s)
    test_sc_df['X_{}'.format(s)] = test_sc_df['Y'].shift(s)
    

# ax = train_sc_df.plot()
# test_sc_df.plot(ax=ax)
# plt.legend(['train_sc.shifted', 'test_sc.shifted'])
# plt.show()

X_train = train_sc_df.dropna().drop('Y', axis=1)
print(X_train)
y_train = train_sc_df.dropna().drop('X_1', axis=1)
print(y_train)


X_test = test_sc_df.dropna().drop('Y', axis=1)
y_test = test_sc_df.dropna().drop('X_1', axis=1)

X_train = X_train.as_matrix()
y_train = y_train.as_matrix()

X_test = X_test.as_matrix()
y_test = y_test.as_matrix()


print('Train size: (%d x %d)'%(X_train.shape[0], X_train.shape[1]))
print('Test size: (%d x %d)'%(X_test.shape[0], X_test.shape[1]))


from sklearn.metrics import r2_score


# ## Build a Neural Network

K.clear_session()
model = Sequential()
model.add(Dense(50, input_shape=(X_test.shape[1],), activation='relu', kernel_initializer='lecun_uniform'))
model.add(Dense(50, input_shape=(X_test.shape[1],), activation='relu'))
model.add(Dense(1))
model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=16, epochs=20, verbose=1)


y_pred = model.predict(X_test)
plt.plot(y_test)
plt.plot(y_pred)
plt.legend(['train', 'test'])
print('R-Squared: %f'%(r2_score(y_test, y_pred)))






Total data: 37 years
80 percent data = 1980 to 2009
                 X_1
Date                
1980-01-03  0.007526
1980-01-04  0.006350
1980-01-07  0.008702
1980-01-08  0.008702
1980-01-09  0.008702
1980-01-10  0.007526
1980-01-11  0.007526
1980-01-14  0.008231
1980-01-15  0.008231
1980-01-16  0.005174
1980-01-17  0.005174
1980-01-18  0.006585
1980-01-21  0.003998
1980-01-22  0.005880
1980-01-23  0.007056
1980-01-24  0.007526
1980-01-25  0.007526
1980-01-28  0.007526
1980-01-29  0.007526
1980-01-30  0.007526
1980-01-31  0.007526
1980-02-01  0.008937
1980-02-04  0.008467
1980-02-05  0.008467
1980-02-06  0.007526
1980-02-07  0.006585
1980-02-08  0.005644
1980-02-11  0.003998
1980-02-13  0.005174
1980-02-14  0.005174
...              ...
2008-11-21  1.000000
2008-11-24  0.984831
2008-11-25  0.994473
2008-11-26  0.989770
2008-11-27  0.980009
2008-11-28  0.967192
2008-12-01  0.985301
2008-12-02  0.992357
2008-12-03  0.998354
2008-12-04  0.994003
2008-12-05  0.990593
2008-12-08  0.983302
200

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:90: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Epoch 1/20
7451/7451 [==============================] - 1s 131us/step - loss: 0.0077
Epoch 2/20
7451/7451 [==============================] - 1s 84us/step - loss: 9.9504e-06
Epoch 3/20
7451/7451 [==============================] - 1s 82us/step - loss: 1.2210e-05
Epoch 4/20
7451/7451 [==============================] - 1s 83us/step - loss: 1.0695e-05
Epoch 5/20
7451/7451 [==============================] - 1s 81us/step - loss: 1.0805e-05
Epoch 6/20
7451/7451 [==============================] - 1s 81us/step - loss: 1.1226e-05
Epoch 7/20
7451/7451 [==============================] - 1s 83us/step - loss: 1.3550e-05
Epoch 8/20
7451/7451 [==============================] - 1s 77us/step - loss: 1.1917e-05
Epoch 9/20
7451/7451 [==============================] - 1s 79us/step - loss: 1.2567e-05
Epoch 10/20
7451/7451 [==============================] - 1s 78us/step - loss: 1.4138e-05
Epoch 11/20
7451/7451 [==============================] - 1s 80us/step - loss: 1.3379e-05
Epoch 12/20
7451/7451 [==========

ValueError: view limit minimum -36827.55 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum -36827.55 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units